In [1]:
import threading
from etl.etl_run import ETLRun

from models import ParentPair, ChildPair

from sqlalchemy import create_engine
from db_constants import Base
from etl.data_loader import DataLoader


engine = create_engine('sqlite:///Cryptos.db', echo=False)
Base.metadata.create_all(engine)

In [2]:
stop_event = threading.Event()
data_loader = DataLoader()
etl = ETLRun(stop_event, pairs_limit=150)
etl.start_threads()

try:
    # Keep the main thread running
    while True:
        pass
except KeyboardInterrupt:
    # On KeyboardInterrupt, stop the WebSocketThread
    stop_event.set()
    etl.stop_threads()


Connected ETHBTC
Connected BTCUSDT
Connected ETHBTC
Connected ETHUSDT
Connected BTCPAX
Connected ETHPAX
Connected BTCTUSD
Connected BTCUSDC
Connected ETHTUSD
Connected ETHBTC
Connected ETHBUSD
Connected ETHBTC
Connected ETHBTC
Connected ETHUSDC
Connected BTCBUSD
Connected ETHBTC
Connected BTCRUB
Connected BTCTRY
Connected ETHBTC
Connected BTCEUR
Connected ETHTRY
Connected ETHBTC
Connected ETHRUB
Connected ETHZAR
Connected BTCBKRW
Connected ETHBTC
Connected BTCGBP
Connected ETHBTC
Connected ETHBKRW
Connected ETHEUR
Connected ETHBTC
Connected BTCZAR
Connected ETHBIDR
Connected ETHBTC
Connected BTCBIDR
Connected ETHBTC
Connected ETHDAI
Connected BTCAUD
Connected BTCNGN
Connected ETHGBP
Connected ETHAUD
Connected ETHBTC
Connected ETHBRL
Connected ETHNGN
Connected ETHBTC
Connected ETHBTC
Connected BTCDAI
Connected ETHUAH
Connected BTCBRL
Connected ETHBTC
Connected BTCUSDP
Connected BTCUAH
Connected ETHUST
Connected BTCPLN
Connected ETHBTC
Connected ETHUSDP
Connected ETHUSDT
Connected USDTBT